In [1]:
import pandas as pd
from tools.api_call import *
%load_ext autoreload
%autoreload 2

## Loading Michelin Data

We first load and demonstate the contents of the [Michelin Guide Restaurants](https://www.kaggle.com/datasets/ngshiheng/michelin-guide-restaurants-2021) dataset, which will be our main data source.

In [7]:
# Load the data
michelin_df = pd.read_csv('michelin_my_maps.csv')
michelin_df["QueryText"] = michelin_df["Name"] + " " + michelin_df["Address"]

In [8]:
# Selected restaurants to demonstrate the API

europe_indexes = [113,83,114,71,106,72,29,54,101,34]
asia_indexes = [139,67,51,79,265]
usa_indexes = [42,43,44,45,46]

all_indexes = europe_indexes + asia_indexes + usa_indexes

In [9]:
# One row of the Michelin data
test_entry = michelin_df.loc[34]
test_entry

Name                                             Enrico Bartolini al Mudec
Address                                via Tortona 56, Milan, 20144, Italy
Location                                                      Milan, Italy
Price                                                                 €€€€
Cuisine                                                           Creative
Longitude                                                         9.161865
Latitude                                                           45.4516
PhoneNumber                                                 390284293701.0
Url                      https://guide.michelin.com/en/lombardia/milano...
WebsiteUrl                                  http://www.enricobartolini.net
Award                                                              3 Stars
FacilitiesAndServices    Air conditioning,Car park,Interesting wine lis...
Description              Enrico Bartolini, who has been awarded more Mi...
QueryText                

# Google Places API
We also plan to extend the data with [Google Places API](https://developers.google.com/maps/documentation/places/web-service/overview?hl=tr), using two methods implemented in `api_call.py`:
* **`search_places(query_text)`**: Returns Google Maps ID from name and address of the restaurant.

* **`get_google_location_details(location_id)`**: Returns the relevant information about the restaurant.

In [10]:
# Retrieve the location id of the restaurant using Places API
location_id = search_places(test_entry["QueryText"])
location_id

'ChIJr54jbebDhkcRlR68yNtO368'

In [21]:
# Retrieve the details of the restaurant using Places API giving the Google location id
google_location_details = get_google_location_details(location_id)

In [15]:
# Opening Hours
google_location_details["currentOpeningHours"]["weekdayDescriptions"]

['Monday: Closed',
 'Tuesday: 12:30\u2009–\u20092:00\u202fPM, 7:30\u2009–\u200910:00\u202fPM',
 'Wednesday: 12:30\u2009–\u20092:00\u202fPM, 7:30\u2009–\u200910:00\u202fPM',
 'Thursday: 12:30\u2009–\u20092:00\u202fPM, 7:30\u2009–\u200910:00\u202fPM',
 'Friday: 12:30\u2009–\u20092:00\u202fPM, 7:30\u2009–\u200910:00\u202fPM',
 'Saturday: 12:30\u2009–\u20092:00\u202fPM, 7:30\u2009–\u200910:00\u202fPM',
 'Sunday: Closed']

In [20]:
# Various details like accessibility options
google_location_details["accessibilityOptions"]

{'wheelchairAccessibleEntrance': True,
 'wheelchairAccessibleRestroom': True,
 'wheelchairAccessibleSeating': True}

In [12]:
# Reviews
google_location_details["reviews"][0]

{'name': 'places/ChIJr54jbebDhkcRlR68yNtO368/reviews/ChZDSUhNMG9nS0VJQ0FnSUNsenBDRFp3EAE',
 'relativePublishTimeDescription': '4 months ago',
 'rating': 5,
 'text': {'text': "Enrico Bartolini's food is amazing! The flavors are unique, and every dish feels like a work of art. The restaurant has a classy vibe, and the service is top-notch. If you love great food, this place is a must-try!",
  'languageCode': 'en'},
 'originalText': {'text': "Enrico Bartolini's food is amazing! The flavors are unique, and every dish feels like a work of art. The restaurant has a classy vibe, and the service is top-notch. If you love great food, this place is a must-try!",
  'languageCode': 'en'},
 'authorAttribution': {'displayName': 'Tze Haw Lo',
  'uri': 'https://www.google.com/maps/contrib/103491011197596279014/reviews',
  'photoUri': 'https://lh3.googleusercontent.com/a/ACg8ocL3NbEh6t0EBNk_d2e1zW7ANwlAUwk9nkjena8VFgTAu58=s128-c0x00000000-cc-rp-mo'},
 'publishTime': '2023-11-18T14:29:32Z'}

In [12]:
# Usage of the API for all the selected restaurants

all_location_details = []
for index in all_indexes:
    test_entry = michelin_df.loc[index]
    location_id = search_places(test_entry["QueryText"])
    google_location_details = get_google_location_details(location_id)
    all_location_details.append(google_location_details)

In [14]:
# Save the data
google_api_df = pd.DataFrame(all_location_details)
google_api_df.to_pickle("google_api_df.pkl")

In [46]:
# Load the data
google_api_df = pd.read_pickle("google_api_df.pkl")

In [60]:
# All columns present in the API response
google_api_df.columns

Index(['name', 'id', 'types', 'nationalPhoneNumber',
       'internationalPhoneNumber', 'formattedAddress', 'addressComponents',
       'plusCode', 'location', 'viewport', 'rating', 'googleMapsUri',
       'websiteUri', 'regularOpeningHours', 'utcOffsetMinutes',
       'adrFormatAddress', 'businessStatus', 'priceLevel', 'userRatingCount',
       'iconMaskBaseUri', 'iconBackgroundColor', 'displayName',
       'primaryTypeDisplayName', 'currentOpeningHours', 'primaryType',
       'shortFormattedAddress', 'editorialSummary', 'reviews', 'photos',
       'goodForChildren', 'accessibilityOptions', 'paymentOptions', 'delivery',
       'dineIn', 'reservable', 'servesBreakfast', 'servesLunch',
       'servesDinner', 'servesBeer', 'servesWine', 'servesBrunch',
       'servesVegetarianFood', 'outdoorSeating', 'liveMusic',
       'menuForChildren', 'servesCocktails', 'servesDessert', 'servesCoffee',
       'restroom', 'goodForGroups', 'goodForWatchingSports', 'parkingOptions',
       'takeout', 'a

# TheFork API
For selected restaurants in Europe, we can also retrieve additional information from [The Fork The Spoon API](https://rapidapi.com/apidojo/api/the-fork-the-spoon), using two methods implemented in `api_call.py`:
* **`get_the_fork_autocomplete(name,latitude,longitude)`**: Returns TheFork autocomplete results and sorts them with respect to their distance to point in (latitude, longitute). We use to TheFork `id` field to feed into another API call.

* **`get_the_fork_data(restaurant_id)`**: Returns TheFork information on the specified restaurant.

In [26]:
# API call to the TheFork API for retrieving TheFork ids using restaurant name and coordinates
the_fork_autocomplete = get_the_fork_autocomplete(test_entry['Name'],
                                                  test_entry["Latitude"],
                                                  test_entry["Longitude"])

In [30]:
# Extracting TheFork id from responses
restaurant_id = the_fork_autocomplete["data"]["autocomplete"][0]["id"]
restaurant_id

'276943'

In [31]:
# API call to TheFork API for retrieving restaurant details using TheFork id
fork_info = get_the_fork_data(restaurant_id)

In [38]:
# Chef name
restaurant_data = fork_info["data"]["restaurant"]
restaurant_data["chefName"]

'Enrico Bartolini '

In [52]:
# Offers in the restaurant
for offer in restaurant_data["offers"][:-5]:
    print(offer["name"])
    print("\t"+offer["description"])

SOLO A PRANZO - 3 portate del giorno €300
	This preset menu includes: 1° PORTATA + 2° PORTATA + 3° PORTATA and drinks.
ONLY FOR LUNCH - 3 courses menù of the day €300
	This preset menu includes: 1° + 2° + 3° and drinks.
CHEESE TASTING €120
	This preset menu includes: 1st + 2nd + 3rd + 4th + 5th
5 ASSAGGI DI FORMAGGI CREATIVI €120
	This preset menu includes: 1° + 2° + 3° + 4° + 5°
MENU A LA CARTE 2 PORTATE A SCELTA €220
	This preset menu includes: Entrée + Dessert
MENU A LA CARTE 2 COURSE OF YOUR CHOICE €220
	This preset menu includes: Entrée + Dessert
MENU A LA CARTE 3 PIATTI A SCELTA €260
	This preset menu includes: Appetizer + Entrée + Dessert
MENU A LA CARTE 3 COURSES OF YOUR CHOICE €260
	This preset menu includes: Appetizer + Entrée + Dessert
MENù A LA CARTE 4 PORTATE A SCELTA €300
	This preset menu includes: Appetizer + Entrée + Entrée + Dessert


In [59]:
# Menu details
for menu in restaurant_data["menus"]:
    print(menu["name"])
    for section in menu["sections"]:
        print(section["name"])
        for dish in section["items"]:
            print("\t"+dish["name"])


À la carte
Dessert a scelta
	Mela e spezie
	Soufflè al guanaja, arachide salato, ananas al rhum, aceto di cocco e sesamo tostato
	Agrumi e agrumi
	Ad un passo dal caffè
 piatti a scelta
	2 piatti a scelta
	3 piatti a scelta
	4 piatti a scelta
SOLO A PRANZO - 3 portate del giorno €300
1° PORTATA
	Proposta dello Chef
2° PORTATA
	Proposta dello Chef
3° PORTATA
	Proposta dello Chef
Drinks
	Acqua, vino e caffè
ONLY FOR LUNCH - 3 courses menù of the day €300
1°
	CHEF'S PROPOSAL
2°
	CHEF'S PROPOSAL
3°
	CHEF'S PROPOSAL
Drinks
	Water, wine and coffee
CHEESE TASTING €120
1st
	Fontina "Estrema di Alpeggio", toasted bread with red fruits, spices, flowers and sour herbs
2nd
	Comtè 24 months, american potato, hazelnut and balsamic vinegar
3rd
	Gorgonzola 250 days, white chocolate and anise
4th
	"Boscasso" goat cheese in ash, its yoghurt, sunflower seeds and 3 flavour herbal tea
5th
	"Storico Ribelle" 2005 and Taggiasca crispy chips
5 ASSAGGI DI FORMAGGI CREATIVI €120
1°
	Fontina estrema di alpeggio,